In [1]:
import json
import os
import requests
import glob
from io import BytesIO
import urllib
from zipfile import ZipFile
from tqdm import tqdm
from datetime import datetime


date_time_str = "2021-01-01T22:50:41"
date_time_format = "%Y-%m-%dT%H:%M:%S"
date_time = datetime.strptime(date_time_str, date_time_format)

maps_dir = "../data/maps"
os.makedirs(maps_dir, exist_ok=True)

def download_map(hash):
    map_dir = f"{maps_dir}/{hash}"

    if os.path.exists(map_dir):
        return
    
    beatsaver_url = f"https://beatsaver.com/api/maps/hash/{hash}"
    r = requests.get(url = beatsaver_url)
    if not r.ok:
        return
    content = r.content.decode('utf-8', errors='replace')
    beatsaver_data = json.loads(content)
    if datetime.strptime(beatsaver_data["uploaded"][:len(date_time_str)], date_time_format) < date_time:
        return
    
    downloadURL = ""
    
    for version in beatsaver_data["versions"]:
        if version["hash"] == hash:
            downloadURL = version["downloadURL"]
    
    if downloadURL == "":
        downloadURL = f"https://r2cdn.beatsaver.com/{hash}.zip"
        # raise Exception()
    
    req = urllib.request.Request(downloadURL, headers={'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"})
    with urllib.request.urlopen(req) as zipresp:
        with ZipFile(BytesIO(zipresp.read())) as zfile:
            zfile.extractall(map_dir)

In [2]:
latest_ranked_url = "https://api.beatleader.xyz/leaderboards?page=1&count=10&type=ranked&sortBy=timestamp&order=desc&allTypes=0&mapRequirements=44&allRequirements=2"
most_plays_unranked = "https://api.beatleader.xyz/leaderboards?page=1&count=2000&type=all&sortBy=playcount&order=desc&allTypes=0&mapRequirements=44&allRequirements=2"
leaderboards = requests.get(most_plays_unranked).json()

In [3]:
for leaderboard in tqdm(leaderboards["data"]):
    try:
        download_map(leaderboard["song"]["hash"].lower())
    except Exception as e:
        print(e)

  5%|▍         | 94/2000 [00:30<07:52,  4.03it/s]

HTTP Error 404: Not Found


 14%|█▍        | 275/2000 [01:26<08:37,  3.33it/s]

HTTP Error 404: Not Found


 19%|█▉        | 389/2000 [02:05<09:55,  2.71it/s]

...

 82%|████████▏ | 1648/2000 [07:05<01:29,  3.93it/s]

HTTP Error 404: Not Found


 88%|████████▊ | 1762/2000 [07:25<00:52,  4.56it/s]

HTTP Error 404: Not Found


100%|██████████| 2000/2000 [08:08<00:00,  4.09it/s]
